# Custom layers

We recommend using [tf.keras](https://www.tensorflow.org/api_docs/python/tf/keras) as a high-level API for building neural networks. That said, most TensorFlow APIs are usable with eager execution.



In [0]:
import tensorflow as tf
tfe = tf.contrib.eager

tf.enable_eager_execution()

# Layers: common sets of useful operations
Most of the time when writing code for machine learning models you want to operate at a higher level of abstraction than individual operations and manipulation of individual variables.

Many machine learning models are expressible as the composition and stacking of relatively simple layers, and TensorFlow provides both a set of many common layers as a well as easy ways for you to write your own application-specific layers either from scratch or as the composition of existing layers.

TensorFlow includes the full [Keras](https://keras.io/) API in the tf.keras package, and the Keras layers are very useful when building your own models.



In [0]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to 
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

The full list of pre-existing layers can be seen in [the documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers). It includes Dense (a fully-connected layer), Conv2D, LSTM, BatchNormalization, Dropout, and many others.

In [3]:
# To use a layer, simply call it.
layer(tf.zeros([10, 5]))

<tf.Tensor: id=30, shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [4]:
# Layers have many useful methods. For example, you can inspect all variables
# in a layer by calling layer.variables. In this case a fully-connected layer
# will have variables for weights and biases.
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-4.0366814e-01,  3.2031327e-01,  5.6271762e-01, -5.8880943e-01,
         -2.8946480e-01,  3.5525382e-01,  1.2851298e-02,  3.7897748e-01,
         -6.0882068e-01, -4.1087961e-01],
        [-3.2560897e-01, -5.6080186e-01,  4.7131866e-01,  3.1059730e-01,
         -1.8097281e-02,  6.0922450e-01,  3.9811134e-03, -4.2806429e-01,
         -3.2269955e-04, -1.1358327e-01],
        [ 1.7385662e-02,  9.9588871e-02,  1.5270764e-01, -5.8298111e-02,
         -4.2097706e-01, -5.1848471e-01, -4.1809669e-01, -2.8321353e-01,
          6.1121279e-01, -1.4117122e-01],
        [ 5.3456753e-01, -5.8435619e-01,  5.5030471e-01,  1.6315055e-01,
         -5.2405035e-01,  1.4842117e-01,  7.6578438e-02, -2.1788701e-01,
         -3.3180127e-01, -3.3434629e-02],
        [ 3.6605895e-02, -4.7766310e-01,  1.8173337e-01,  1.8173593e-01,
         -3.6914390e-01, -2.6182982e-01, -6.3673139e-02, -4.3127006e-01,
         -2.6092434e-01, -5.22282

In [5]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-4.0366814e-01,  3.2031327e-01,  5.6271762e-01, -5.8880943e-01,
         -2.8946480e-01,  3.5525382e-01,  1.2851298e-02,  3.7897748e-01,
         -6.0882068e-01, -4.1087961e-01],
        [-3.2560897e-01, -5.6080186e-01,  4.7131866e-01,  3.1059730e-01,
         -1.8097281e-02,  6.0922450e-01,  3.9811134e-03, -4.2806429e-01,
         -3.2269955e-04, -1.1358327e-01],
        [ 1.7385662e-02,  9.9588871e-02,  1.5270764e-01, -5.8298111e-02,
         -4.2097706e-01, -5.1848471e-01, -4.1809669e-01, -2.8321353e-01,
          6.1121279e-01, -1.4117122e-01],
        [ 5.3456753e-01, -5.8435619e-01,  5.5030471e-01,  1.6315055e-01,
         -5.2405035e-01,  1.4842117e-01,  7.6578438e-02, -2.1788701e-01,
         -3.3180127e-01, -3.3434629e-02],
        [ 3.6605895e-02, -4.7766310e-01,  1.8173337e-01,  1.8173593e-01,
         -3.6914390e-01, -2.6182982e-01, -6.3673139e-02, -4.3127006e-01,
         -2.6092434e-01, -5.22282

# Implementing custom layers
The best way to implement your own layer is extending the tf.keras.Layer class and implementing: * `__init__ `, where you can do all input-independent initialization * `build`, where you know the shapes of the input tensors and can do the rest of the initialization * `call`, where you do the forward computation

Note that you don't have to wait until `build` is called to create your variables, you can also create them in `__init__`. However, the advantage of creating them in build is that it enables late variable creation based on the shape of the inputs the layer will operate on. On the other hand, creating variables in `__init__` would mean that shapes required to create the variables will need to be explicitly specified.

In [6]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs
    
  def build(self, input_shape):
    self.kernel = self.add_variable("kernel", 
                                    shape=[input_shape[-1].value, 
                                           self.num_outputs])
    
  def call(self, input):
    return tf.matmul(input, self.kernel)
  
layer = MyDenseLayer(10)
print(layer(tf.zeros([10, 5])))
print(layer.variables)

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(10, 10), dtype=float32)
[<tf.Variable 'my_dense_layer/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[-0.48930794, -0.3148687 ,  0.29035568, -0.37791938,  0.02816045,
         0.27328676,  0.11643714,  0.2509973 ,  0.02953011,  0.22730422],
       [-0.10154003, -0.5281738 , -0.29052424,  0.23391211,  0.37096137,
         0.18079996,  0.43431705, -0.23190239,  0.25416398,  0.2310248 ],
       [-0.572566  , -0.37430662,  0.607472  ,  0.43998986,  0.20894265,
        -0.12685758,  0.01026618,  0.03067052,  0.3161133 ,  0.06499451],
       [-0.05167484,  0.51854426, -0.42793733, -0.5962794 , -0.30030248,
        -0.06430477,  0.4917336 ,  0

Note that you don't have to wait until `build` is called to create your variables, you can also create them in `__init__`.

Overall code is easier to read and maintain if it uses standard layers whenever possible, as other readers will be familiar with the behavior of standard layers. If you want to use a layer which is not present in tf.keras.layers or tf.contrib.layers, consider filing a [github issue](http://github.com/tensorflow/tensorflow/issues/new) or, even better, sending us a pull request!



# Models: composing layers
Many interesting layer-like things in machine learning models are implemented by composing existing layers. For example, each residual block in a resnet is a composition of convolutions, batch normalizations, and a shortcut.

The main class used when creating a layer-like thing which contains other layers is tf.keras.Model. Implementing one is done by inheriting from tf.keras.Model.



In [7]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)

    
block = ResnetIdentityBlock(1, [1, 2, 3])
print(block(tf.zeros([1, 2, 3, 3])))
print([x.name for x in block.variables])

tf.Tensor(
[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]], shape=(1, 2, 3, 3), dtype=float32)
['resnet_identity_block/conv2d/kernel:0', 'resnet_identity_block/conv2d/bias:0', 'resnet_identity_block/batch_normalization/gamma:0', 'resnet_identity_block/batch_normalization/beta:0', 'resnet_identity_block/conv2d_1/kernel:0', 'resnet_identity_block/conv2d_1/bias:0', 'resnet_identity_block/batch_normalization_1/gamma:0', 'resnet_identity_block/batch_normalization_1/beta:0', 'resnet_identity_block/conv2d_2/kernel:0', 'resnet_identity_block/conv2d_2/bias:0', 'resnet_identity_block/batch_normalization_2/gamma:0', 'resnet_identity_block/batch_normalization_2/beta:0', 'resnet_identity_block/batch_normalization/moving_mean:0', 'resnet_identity_block/batch_normalization/moving_variance:0', 'resnet_identity_block/batch_normalization_1/moving_mean:0', 'resnet_identity_block/batch_normalization_1/moving_variance:0', 'resnet_identity_block/batch_normalization_2

Much of the time, however, models which compose many layers simply call one layer after the other. This can be done in very little code using tf.keras.Sequential



In [8]:
 my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1)),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Conv2D(2, 1, 
                                                      padding='same'),
                               tf.keras.layers.BatchNormalization(),
                               tf.keras.layers.Conv2D(3, (1, 1)),
                               tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: id=503, shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

**Next steps**

Now you can go back to the previous notebook and adapt the linear regression example to use layers and models to be better structured.

